In [50]:
import os
import numpy as np
import json
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, InputLayer, Flatten

In [51]:
# Get the current working directory
CURRENT_DIRECTORY = os.getcwd()
# Get the parent directory
PARENT_DIRECTORY = os.path.dirname(CURRENT_DIRECTORY)
PARENT_DIRECTORY = os.path.dirname(PARENT_DIRECTORY)

# Open the config file and load its content into a dictionary
config_file = open(PARENT_DIRECTORY + '\\config\\config.json')
CONFIG_DATA = json.load(config_file)

# Close the file after loading the data
config_file.close()

In [52]:
WINDOW_SIZE = CONFIG_DATA['WINDOW_SIZE']
NUMBER_OF_FEATURES = CONFIG_DATA["NUMBER_OF_FEATURES"]
NUM_EPOCHS = CONFIG_DATA["NUMBER_OF_EPOCHS"]
CLASSICATIONS = CONFIG_DATA['CLASSES_MOTIONSPEED']

ALL_X_TRAINING_DATA_PATH = PARENT_DIRECTORY + '\\NeuralNetwork\\np-saved-data\\training\\ALL-X-TRAIN-CLASSIFY-MOTIONSPEED-SML-STEPS-V0.npy'
ALL_Y_TRAINING_DATA_PATH = PARENT_DIRECTORY + '\\NeuralNetwork\\np-saved-data\\training\\ALL-Y-TRAIN-CLASSIFY-MOTIONSPEED-SML-STEPS-V0.npy'

ALL_X_TEST_DATA_PATH = PARENT_DIRECTORY + '\\NeuralNetwork\\np-saved-data\\test\\ALL-X-TEST-CLASSIFY-MOTIONSPEED-SML-STEPS-V0.npy'
ALL_Y_TEST_DATA_PATH = PARENT_DIRECTORY + '\\NeuralNetwork\\np-saved-data\\test\\ALL-Y-TEST-CLASSIFY-MOTIONSPEED-SML-STEPS-V0.npy'

MOTION_MODEL_SAVE_PATH =  PARENT_DIRECTORY + '\\NeuralNetwork\\models\\standard-backprop-motionspeed-sml-steps-2.h5'

In [53]:
ALL_X_TRAIN_DATA = np.load(ALL_X_TRAINING_DATA_PATH)
ALL_Y_TRAIN_DATA = np.load(ALL_Y_TRAINING_DATA_PATH)

ALL_X_TEST_DATA = np.load(ALL_X_TEST_DATA_PATH)
ALL_Y_TEST_DATA = np.load(ALL_Y_TEST_DATA_PATH)

ALL_X_TRAIN_DATA = ALL_X_TRAIN_DATA[:, [0, 2, 4, 6]]
ALL_X_TEST_DATA = ALL_X_TEST_DATA[:, [0, 2, 4, 6]]

# Sum between columns 0 and 1
sum_cols_01 = np.sum(ALL_X_TRAIN_DATA[:, 0:2], axis=1)

# Sum between columns 2 and 3
sum_cols_23 = np.sum(ALL_X_TRAIN_DATA[:, 2:4], axis=1)

ALL_X_TRAIN_DATA =  np.column_stack((sum_cols_01, sum_cols_23))

# Sum between columns 0 and 1
sum_cols_01 = np.sum(ALL_X_TEST_DATA[:, 0:2], axis=1)
# Sum between columns 2 and 3
sum_cols_23 = np.sum(ALL_X_TEST_DATA[:, 2:4], axis=1)

ALL_X_TEST_DATA =  np.column_stack((sum_cols_01, sum_cols_23))

print(ALL_X_TRAIN_DATA.shape)
print(ALL_Y_TRAIN_DATA.shape)

print(ALL_X_TEST_DATA.shape)
print(ALL_Y_TEST_DATA.shape)

(46587, 2)
(46587,)
(53071, 2)
(53071,)


In [54]:
# Generate random permutation indices
random_indices_1 = np.random.permutation(len(ALL_X_TRAIN_DATA))


# Shuffle the input features and target labels using the random indices
shuffled_X_TRAIN_DATA = ALL_X_TRAIN_DATA[random_indices_1]
shuffled_Y_TRAIN_DATA = ALL_Y_TRAIN_DATA[random_indices_1]

In [55]:
# Generate random permutation indices
random_indices_2 = np.random.permutation(len(ALL_X_TEST_DATA))

# Shuffle the input features and target labels using the random indices
shuffled_X_TEST_DATA = ALL_X_TEST_DATA[random_indices_2]
shuffled_Y_TEST_DATA = ALL_Y_TEST_DATA[random_indices_2]

In [56]:
output_activation_function = 'sigmoid' if len(CLASSICATIONS) == 2 else 'softmax'
loss_algorithm = 'binary_crossentropy' if len(CLASSICATIONS) == 2 else 'sparse_categorical_crossentropy'
output_size = 1 if len(CLASSICATIONS) == 2 else len(CLASSICATIONS)

In [57]:
model2_motion = Sequential()
model2_motion.add(InputLayer((2)))
model2_motion.add(Dense(8, activation='relu'))
model2_motion.add(Dense(4, activation='relu'))
model2_motion.add(Dense(output_size, activation=output_activation_function))

model2_motion.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_15 (Dense)            (None, 8)                 24        
                                                                 
 dense_16 (Dense)            (None, 4)                 36        
                                                                 
 dense_17 (Dense)            (None, 3)                 15        
                                                                 
Total params: 75
Trainable params: 75
Non-trainable params: 0
_________________________________________________________________


In [58]:
# Compile the model
model2_motion.compile(optimizer='adam', loss=loss_algorithm, metrics=['acc'])

history2_motion = model2_motion.fit(shuffled_X_TRAIN_DATA, shuffled_Y_TRAIN_DATA, validation_data=(shuffled_X_TEST_DATA, shuffled_Y_TEST_DATA), epochs=NUM_EPOCHS)

# Save the model to a file
model2_motion.save(MOTION_MODEL_SAVE_PATH)

Epoch 1/12
1456/1456 [==============================] - 6s 3ms/step - loss: 0.2618 - acc: 0.9286 - val_loss: 0.3539 - val_acc: 0.8327
Epoch 2/12
1456/1456 [==============================] - 6s 4ms/step - loss: 0.1011 - acc: 0.9670 - val_loss: 0.5252 - val_acc: 0.8024
Epoch 3/12
1456/1456 [==============================] - 4s 3ms/step - loss: 0.0966 - acc: 0.9680 - val_loss: 0.4350 - val_acc: 0.8415
Epoch 4/12
1456/1456 [==============================] - 5s 3ms/step - loss: 0.0973 - acc: 0.9677 - val_loss: 0.4705 - val_acc: 0.8276
Epoch 5/12
1456/1456 [==============================] - 5s 3ms/step - loss: 0.0961 - acc: 0.9677 - val_loss: 0.4671 - val_acc: 0.8339
Epoch 6/12
1456/1456 [==============================] - 5s 3ms/step - loss: 0.0960 - acc: 0.9681 - val_loss: 0.4624 - val_acc: 0.8352
Epoch 7/12
1456/1456 [==============================] - 5s 3ms/step - loss: 0.0951 - acc: 0.9681 - val_loss: 0.4993 - val_acc: 0.8272
Epoch 8/12
1456/1456 [==============================] - 5s 3ms